In [27]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import PoissonRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import r_regression
from sklearn import neighbors
from sklearn import neural_network
from sklearn import tree
import sklearn.ensemble as se
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn import gaussian_process
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv('finalDatasetNew.csv')
df['avgSeverity'] = df['severity']/df['cnt']

asList = []

for i in df['avgSeverity']:
    temp = 'risk'
    if i > 0:
        temp = 'risk'
    else:
        temp = 'no risk'
    asList.append(temp)
    
df['risk'] = asList

In [3]:
print(len(df['risk'][df['risk']=='risk']),len(df['risk']))

8548 47650


In [4]:
dfCat = df[['region','temperature','dayNight','precip','traffic','risk']].copy()

dfCat.region = dfCat.region.astype('category')
dfCat.temperature = dfCat.temperature.astype('category')
# dfCat.dewPoint = dfCat.dewPoint.astype('category')
dfCat.dayNight = dfCat.dayNight.astype('category')
dfCat.precip = dfCat.precip.astype('category')
dfCat.traffic = dfCat.traffic.astype('category')
dfCat.risk = dfCat.risk.astype('category')

In [5]:
dfTransformed = df[['lat','lon']].copy()
dfTransformed.head()

,lat,lon
0,36.99,-113.67
1,36.99,-113.67
2,36.99,-113.67
3,36.99,-113.67
4,36.99,-113.67


In [6]:
# dfTransformed.severity
# dfTransformed.severity = dfTransformed.severity.astype(float)
# dfTransformed.count = dfTransformed.cnt.astype(float)
dfTransformed.lat = dfTransformed.lat.astype(float)
dfTransformed.lon = dfTransformed.lon.astype(float)

In [7]:
for i in dfTransformed.iteritems():
    name = i[0]
    try:
        #print(name)
        skew = dfTransformed[name].skew()
        #print('Regular skew:',name, skew)
        ln = np.log1p(dfTransformed[name] + abs(min(dfTransformed[name])) + 4)
        lnSkew = abs(ln.skew())
        #print('ln skew:', lnSkew)
        sr = (4 + dfTransformed[name])**(1/2)
        srSkew = abs(sr.skew())
        #print('sr skew:', srSkew)
        cr = (4 + dfTransformed[name])**(1/3)
        crSkew = abs(cr.skew())
        #print('cr skew:', crSkew)
        recip = 1/(4 + dfTransformed[name])
        recScew = abs(recip.skew())
        type = 'original'
        finalSkew = skew
        skews = [['log',lnSkew],['squareRoot',srSkew],['cubeRoot',crSkew],['recip',recScew]]
        for ii in skews:
            if ii[1] < finalSkew:
                if (ii[0] == 'log') and (ln.isna().sum() == 0):
                    finalSkew = ii[1]
                    type = ii[0]
                elif (ii[0] == 'squareRoot') and (sr.isna().sum() == 0):
                    finalSkew = ii[1]
                    type = ii[0]
                elif (ii[0] == 'cubeRoot') and (cr.isna().sum() == 0):
                    finalSkew = ii[1]
                    type = ii[0]
                elif (ii[0] == 'recip') and (recip.isna().sum() == 0):
                    finalSkew = ii[1]
                    type = ii[0]
        #print('{} has {} regular skew, {} log skew, {} sqrt skew, and {} cube skew.'.format(name,skew,lnSkew,srSkew,crSkew))
        print('{} will use {} transformation with final skew: {}.'.format(name,type,abs(finalSkew)))
        if type == 'log':
            dfTransformed[name] = ln
        elif type == 'squareRoot':
            dfTransformed[name] = sr
        elif type == 'cubeRoot':
            dfTransformed[name] = cr
        elif type == 'recip':
            dfTransformed[name] = recip
    except:
        pass

lat will use original transformation with final skew: 0.05254324672054759.
lon will use log transformation with final skew: 0.10525802405777833.


In [8]:
print(abs(df.skew()))
print(abs(dfTransformed.skew()))

lat             0.052543
lon             0.456013
precip          0.004869
traffic         0.004617
severity       24.961830
cnt            24.961266
avgSeverity     3.508830
dtype: float64
lat    0.052543
lon    0.105258
dtype: float64


C:\Users\Tony Anderson\AppData\Local\Temp\ipykernel_11316\2999164132.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(abs(df.skew()))


In [9]:
scaler = MinMaxScaler()

In [10]:
dfScaled = scaler.fit_transform(dfTransformed.to_numpy())
dfScaled = pd.DataFrame(dfScaled, columns = ['lat','lon'])

In [11]:
dfFinal = dfCat.join(dfScaled)
print(len(dfFinal))
# dfFinal['avgSeverity'] = dfFinal['severity']/dfFinal['cnt']
dfFinal = dfFinal.dropna()

47650


In [12]:
dfFinal.head()

,region,temperature,dayNight,precip,traffic,risk,lat,lon
0,"36.99,-113.67",< 80,Night,True,True,no risk,0.0,0.094912
1,"36.99,-113.67",80+,Night,False,True,no risk,0.0,0.094912
2,"36.99,-113.67",80+,Night,True,False,no risk,0.0,0.094912
3,"36.99,-113.67",80+,Night,True,True,no risk,0.0,0.094912
4,"36.99,-113.67",80+,Day,False,False,no risk,0.0,0.094912


In [13]:
records = len(dfFinal)
testSize = 4000
trainSize = records - testSize
print(trainSize)

dfFinal = dfFinal.sample(frac=1)

dfTrain = dfFinal.head(trainSize)
dfTest = dfFinal.tail(testSize)

43520


In [14]:
dfTrain.head()

,region,temperature,dayNight,precip,traffic,risk,lat,lon
15179,"38.65,-109.55",80+,Night,True,False,no risk,0.332,0.919912
36711,"40.61,-111.23",<= 32,Night,True,False,no risk,0.724,0.637842
33346,"40.37,-110.77",80+,Day,True,False,no risk,0.676,0.720716
34120,"40.41,-111.45",<= 32,Night,False,False,no risk,0.684,0.596458
29346,"40.11,-111.67",80+,Day,True,True,no risk,0.624,0.553852


In [15]:
yTrain = dfTrain.risk.copy()
xTrain = dfTrain.copy()
xTrain = xTrain.drop(columns = ['region','risk'])
xTrain = pd.get_dummies(xTrain, columns = ['temperature','dayNight','precip','traffic'])
yTest = dfTest.risk.copy()
yTest = yTest.copy()
xTest = dfTest.copy().drop(columns = ['region','risk'])
xTest = pd.get_dummies(xTest, columns = ['temperature','dayNight','precip','traffic'])

In [16]:
yTrain.head()

15179    no risk
36711    no risk
33346    no risk
34120    no risk
29346    no risk
Name: risk, dtype: category
Categories (2, object): ['no risk', 'risk']

In [17]:
rs = 12445

AdaBoostModel = se.AdaBoostClassifier(random_state = rs)
BaggingModel = se.BaggingClassifier(random_state = rs)
ExtraTreesModel = se.ExtraTreesClassifier(random_state = rs)
GradientBoostingModel = se.GradientBoostingClassifier(random_state = rs)
RandomForestModel = se.RandomForestClassifier(random_state = rs)
#LinearRegressionModel = LinearRegression()
#LassoModel = Lasso(random_state = rs)
#PoissonModel = PoissonRegressor()
KNNModel = neighbors.KNeighborsClassifier()
NeuralNetworkModel = neural_network.MLPClassifier(random_state = rs)
DecisionTreeModel = tree.DecisionTreeClassifier()

In [76]:
def cross_validation(_model, _X, _y, _cv): 
    _scoring = ['accuracy']
    results = cross_validate(estimator = _model,
              X = _X,
              y = _y,
              cv = _cv,
              scoring  = _scoring,               
              return_train_score = False)
    return results['accuracy'].mean()

def modelEvaluation(_model, _X, _y, _cv):
    _scoring = {'recall': make_scorer(recall_score, average = 'macro')}
    reg = GridSearchCV(
        estimator = models[_model]['model'],
        param_grid = models[_model]['params'],
        cv = _cv,
        n_jobs = 5,
        scoring = _scoring,
        refit = 'recall'
    )
    reg.fit(_X,_y)
    print('The {} model had a best recall of {} using the following parameters: {}.'.format(_model, reg.best_score_, reg.best_params_))
    
def modelTesting(_model, _xTr, _yTr, _xTe, _yTe):
    _model.fit(_xTr,_yTr)
    preds = _model.predict(_xTe)
    accuracy = metrics.accuracy_score(preds,_yTe)
    cm = metrics.confusion_matrix(_yTe,preds)
    tp = cm[1,1]
    fp = cm[1,0]
    tn = cm[0,0]
    fn = cm[0,1]
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    y_pred_proba = _model.predict_proba(_xTe)[::,1]
    auc = metrics.roc_auc_score(_yTe, y_pred_proba)
    print('The model had an overall:\n -auc of {} \n -accuracy of {}\n -precision of {}\n -sensitivity of {}\n -specificity of {}.'.format(auc, accuracy,precision,sensitivity,specificity))
    
def feature_importances(model,name):
    try:
        for i in range(0,len(model.feature_names_in_)):
            feature_name = model.feature_names_in_[i]
            feature_importance = model.feature_importances_[i]
            temp0 = [name,feature_name,feature_importance]
            print(temp)
    except:
        print(name)

In [162]:
models = {'RandomForest':{'model':RandomForestModel, 'params':{'n_estimators' : list(range(1800,2500,200)), 'min_samples_split' : [2,5], 'min_samples_leaf' : [1,2]}}
         #,'LinearRegression': {'model':LinearRegressionModel, 'params':{}}
         #,'Lasso': {'model':LassoModel, 'params':{'alpha': [0.1,0.01,0.001,0.0001], 'max_iter': list(range(10,100,20))}}
         ,'NeuralNetwork': {'model':NeuralNetworkModel, 'params': {'hidden_layer_sizes':list(range(206,215,1)), 'alpha':[0.01]}}
         ,'KNN': {'model':neighbors.KNeighborsClassifier(), 'params':{'n_neighbors': [17,18,19,20,21,22]}}
         ,'GradientBoosting': {'model': GradientBoostingModel, 'params': {'n_estimators': [500,750,50], 'min_samples_split': [15], 'min_samples_leaf': [1]}}
         ,'AdaBoosting': {'model': AdaBoostModel, 'params': {'n_estimators': list(range(5,15,1))}}
         ,'Bagging': {'model': BaggingModel, 'params': {'n_estimators': [41], 'max_samples': [67,68,69,70,71,72,73]}}
         ,'ExtraTrees': {'model': ExtraTreesModel, 'params': {'n_estimators': [430], 'min_samples_split': [3,5,7]}}
         #,'Poisson': {'model': PoissonModel, 'params': {'alpha': [0.001], 'max_iter': list(range(10,30,2))}}
         ,'DecisionTree': {'model': DecisionTreeModel, 'params': {'min_samples_split': [2], 'min_samples_leaf': [7,8,9,10,11,12]}}}

<h2>Random Forest (Tuned)</h2>

In [ ]:
modelEvaluation('RandomForest',xTrain,yTrain,5)

In [ ]:
trainedRF = se.RandomForestRegressor(random_state = rs, min_samples_leaf = 2, min_samples_split = 5, n_estimators=2000)
trainedRF.fit(xTrain,yTrain)
preds = trainedRF.predict(xTest)
rmse = metrics.mean_squared_error(preds,yTest)
print(rmse)
r2 = metrics.r2_score(preds,yTest)
print(r2)

<h2>Linear Regression (Depricated)</h2>

In [20]:
modelEvaluation('LinearRegression',xTrain,yTrain,5)

KeyError: 'LinearRegression'

In [ ]:
trainedLR = LinearRegression()
trainedLR.fit(xTrain,yTrain)
preds = trainedLR.predict(xTest)
rmse = metrics.mean_squared_error(preds,yTest)
print(rmse)
r2 = metrics.r2_score(preds,yTest)
print(r2)

<h2>Lasso (Depricated)</2>

In [ ]:
modelEvaluation('Lasso',xTrain,yTrain,5)

In [ ]:
trainedLassoR = Lasso(alpha = 0.0001, max_iter = 10000)
trainedLassoR.fit(xTrain,yTrain)
preds = trainedLassoR.predict(xTest)
rmse = metrics.mean_squared_error(preds,yTest)
print(rmse)
r2 = metrics.r2_score(preds,yTest)
print(r2)

<h2>Neural Network (Tuned)</h2>

In [ ]:
modelEvaluation('NeuralNetwork',xTrain,yTrain,5)

In [173]:
trainedNN = neural_network.MLPClassifier(alpha = 0.01, hidden_layer_sizes = 209, random_state = rs)
modelTesting(trainedNN, xTrain, yTrain, xTest, yTest)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


The model had an overall:
 -auc of 0.9156871164324516 
 -accuracy of 0.87125
 -precision of 0.4716981132075472
 -sensitivity of 0.7399577167019028
 -specificity of 0.8888573858803516.


In [174]:
try:
    for i in range(0,len(trainedNN.feature_names_in_)):
        feature_name = trainedNN.feature_names_in_[i]
        feature_importance = trainedNN.feature_importances_[i]
        temp0 = ['NN',feature_name,feature_importance]
        print(temp0[1])
except:
    print('NN')

NN


<h2>K-Nearest Neighbors (Re-Tuned)</h2>



In [ ]:
modelEvaluation('KNN',xTrain,yTrain,5)

In [78]:
trainedKNN = neighbors.KNeighborsClassifier(n_neighbors = 17)
modelTesting(trainedKNN, xTrain, yTrain, xTest, yTest)

The model had an overall:
 -auc of 0.9242869304502787 
 -accuracy of 0.89275
 -precision of 0.6725067385444744
 -sensitivity of 0.7284671532846715
 -specificity of 0.9266968325791856.


In [43]:
try:
    for i in range(0,len(trainedKNN.feature_names_in_)):
        feature_name = trainedKNN.feature_names_in_[i]
        feature_importance = trainedKNN.feature_importances_[i]
        temp0 = ['KNN',feature_name,feature_importance]
        print(temp0[1])
except:
    print('KNN')

KNN


<h2>Gradient Boosting (Re-Tuned)</h2>

In [ ]:
modelEvaluation('GradientBoosting',xTrain,yTrain,5)

In [79]:
trainedGB = se.GradientBoostingClassifier(random_state = rs, min_samples_split = 40, n_estimators = 650, min_samples_leaf = 1)
modelTesting(trainedGB, xTrain, yTrain, xTest, yTest)

The model had an overall:
 -auc of 0.938302192901901 
 -accuracy of 0.894
 -precision of 0.6185983827493261
 -sensitivity of 0.765
 -specificity of 0.9167647058823529.


In [71]:
try:
    for i in range(0,len(trainedGB.feature_names_in_)):
        feature_name = trainedGB.feature_names_in_[i]
        feature_importance = trainedGB.feature_importances_[i]
        temp0 = ['Gradient Boosting',feature_name,feature_importance]
        print(temp0[2])
except:
    print('Gradient Boosting')

0.16615110737249472
0.16831066899531755
0.0365244867393739
0.0444090084136359
0.010155343383198557
0.014395459054785072
0.006894755453537663
0.14867530221041284
0.10563554672866642
0.2098024111492393
0.08904591049933828


<h2>Ada Boosting (Re-Tuned)</h2>

In [92]:
modelEvaluation('AdaBoosting',xTrain,yTrain,5)

The AdaBoosting model had a best recall of 0.6718005828031173 using the following parameters: {'n_estimators': 13}.


In [93]:
trainedAB = se.AdaBoostClassifier(random_state = rs, n_estimators = 13)
modelTesting(trainedAB, xTrain, yTrain, xTest, yTest)

The model had an overall:
 -auc of 0.8804071752054656 
 -accuracy of 0.85225
 -precision of 0.4029649595687331
 -sensitivity of 0.668903803131991
 -specificity of 0.8753166338305657.


In [95]:
try:
    for i in range(0,len(trainedAB.feature_names_in_)):
        feature_name = trainedAB.feature_names_in_[i]
        feature_importance = trainedAB.feature_importances_[i]
        temp0 = ['Ada Boosting',feature_name,feature_importance]
        print(temp0[1])
except:
    print('Nope')

lat
lon
temperature_80+
temperature_< 80
temperature_<= 32
dayNight_Day
dayNight_Night
precip_False
precip_True
traffic_False
traffic_True


<h2>Bagging (Re-Tuned)</h2>

In [109]:
modelEvaluation('Bagging',xTrain,yTrain,5)

The Bagging model had a best recall of 0.6779611652026765 using the following parameters: {'max_samples': 68, 'n_estimators': 41}.


In [110]:
trainedB = se.BaggingClassifier(random_state = rs, n_estimators = 41, max_samples = 68)
modelTesting(trainedB, xTrain, yTrain, xTest, yTest)

The model had an overall:
 -auc of 0.8779187949546544 
 -accuracy of 0.85825
 -precision of 0.42857142857142855
 -sensitivity of 0.6898047722342733
 -specificity of 0.8801921446736366.


In [176]:
try:
    for i in range(0,len(trainedB.feature_names_in_)):
        feature_name = trainedB.feature_names_in_[i]
        feature_importance = trainedB.feature_importances_[i]
        temp0 = ['B',feature_name,feature_importance]
        print(temp0[1])
except:
    print('B')

B


<h2>Extra Trees (Re-Tuned)</h2>

In [147]:
modelEvaluation('ExtraTrees',xTrain,yTrain,5)

The ExtraTrees model had a best recall of 0.7916975378799803 using the following parameters: {'min_samples_split': 5, 'n_estimators': 430}.


In [148]:
trainedET = se.ExtraTreesClassifier(random_state = rs, n_estimators = 430, min_samples_split = 5)
modelTesting(trainedET, xTrain, yTrain, xTest, yTest)

The model had an overall:
 -auc of 0.9279132932578152 
 -accuracy of 0.89675
 -precision of 0.6752021563342318
 -sensitivity of 0.7444279346210996
 -specificity of 0.9275623685001503.


In [179]:
try:
    for i in range(0,len(trainedET.feature_names_in_)):
        feature_name = trainedET.feature_names_in_[i]
        feature_importance = trainedET.feature_importances_[i]
        temp0 = ['Extra Trees',feature_name,feature_importance]
        print(temp0[1])
except:
    print('Nope')

lat
lon
temperature_80+
temperature_< 80
temperature_<= 32
dayNight_Day
dayNight_Night
precip_False
precip_True
traffic_False
traffic_True


<h2>Poisson (Depricated)</h2>

In [ ]:
modelEvaluation('Poisson',xTrain,yTrain,5)

In [ ]:
trainedP = PoissonRegressor(alpha = 0.001, max_iter=19)
trainedP.fit(xTrain,yTrain)
preds = trainedP.predict(xTest)
rmse = metrics.mean_squared_error(preds,yTest)
print(rmse)
r2 = metrics.r2_score(preds,yTest)
print(r2)

<h2>Decision Tree (Re-Tuned)</h2>

In [163]:
modelEvaluation('DecisionTree',xTrain,yTrain,5)

The DecisionTree model had a best recall of 0.788170903333968 using the following parameters: {'min_samples_leaf': 9, 'min_samples_split': 2}.


In [164]:
trainedDT = tree.DecisionTreeClassifier(min_samples_leaf=9, min_samples_split=2)
modelTesting(trainedDT, xTrain, yTrain, xTest, yTest)

The model had an overall:
 -auc of 0.9171533393231507 
 -accuracy of 0.897
 -precision of 0.6671159029649596
 -sensitivity of 0.75
 -specificity of 0.9260479041916168.


In [178]:
try:
    for i in range(0,len(trainedDT.feature_names_in_)):
        feature_name = trainedDT.feature_names_in_[i]
        feature_importance = trainedDT.feature_importances_[i]
        temp0 = ['DT',feature_name,feature_importance]
        print(temp0[2])
except:
    print('DT')

0.28986282643682104
0.22878386381873386
0.022696040037638684
0.038686318816189405
0.01238735706415529
0.007103480102031717
0.01747956981449471
0.1609660879679341
0.0
0.01018433861423617
0.21185011732776504


In [ ]:
fi = []

def feature_importances(model,name):
    try:
        for i in range(0,len(model.feature_names_in_)):
            feature_name = model.feature_names_in_[i]
            feature_importance = model.feature_importances_[i]
            temp0 = [name,feature_name,feature_importance]
            fi.append(temp0)
    except:
        fi.append([name,'',''])
        
feature_importances(trainedRF,'Random Forest')
feature_importances(trainedLR, 'Linear Regression')
feature_importances(trainedLassoR, 'Lasso')
feature_importances(trainedNN, 'Neural Network')
feature_importances(trainedKNN, 'K-Nearest Neighbors')
feature_importances(trainedGB, 'Gradient Boosting')
feature_importances(trainedAB, 'Ada Boosting')
feature_importances(trainedB, 'Bagging')
feature_importances(trainedET, 'Extra Trees')
feature_importances(trainedP, 'Poisson')
feature_importances(trainedDT, 'Decision Tree')

#print(fi)

# trainedRF.feature_importances_
# trainedRF.feature_names_in_

In [ ]:
for i in fi:
    print(i[0])